In [11]:
from decouple import config
import requests
import json
from pathlib import Path
from pymongo import MongoClient
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MultiLabelBinarizer
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
from multiprocessing import Process, Pool
import multiprocessing 
from pymongo import UpdateOne, InsertOne
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import chain
import re
from datetime import datetime
from scipy.stats.mstats import winsorize
from scipy.stats import norm
import warnings
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from joblib import Parallel, delayed
from sklearn import metrics
from collections import defaultdict, Counter
import itertools
from deep_translator import GoogleTranslator
import langdetect
from difflib import SequenceMatcher
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import shuffle
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm


In [3]:
%%script false
nltk.download('punkt')

Couldn't find program: 'false'


In [33]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['DM_Project']
collection = db['Games_updated']
df = pd.DataFrame(list(collection.find()))

In [5]:
# Add 1 to 0 values and apply log
def apply_log(Attr):
    Attr = np.where(Attr == 0, Attr + 1, Attr)
    #Apply log
    return np.log(Attr)

# Winsorization of the dataset

In [7]:
#Apply Winsorization
df['Net Votes'] = (winsorize(df['Net Votes'], limits=[0, 0.001]))
df['Peak CCU'] = (winsorize(df['Peak CCU'], limits=[0, 0.001]))

# Combining Genres and Categories

In [8]:
#Turn string lists from kaggle dataset into arrays
df['Genres'] = df['Genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
df['Categories'] = df['Categories'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
df['Genres'] = df['Genres'].apply(lambda x: x if isinstance(x, list) else [])
#Combining Categories and Genres
df['Combined Tags'] = df['Genres']+df['Categories']
#Nan or 0.0 values into empty lists
df['Combined Tags'] = df['Combined Tags'].apply(lambda x: x if isinstance(x, list) else [])

#### Replace separators with space in df tags 

In [9]:
combine_df = df.copy()
from tqdm.auto import tqdm
tqdm.pandas(desc="Processing Tags")

#Even with l=en in the API call some tags are in native language, Translate tags to english
def translate_tags(tags):
    translated_tags = []
    for t in tags:
        translated_text = GoogleTranslator(source='auto', target='en').translate(t)
        #Single is translated to one for most languages, replace it
        translated_text = translated_text.replace("one ","single ")
        if translated_text not in translated_tags:
            translated_tags.append(translated_text)
    return translated_tags

# Util to replace a character with space
def remove_chars(s, rem):
    for c in rem:
        s = s.replace(c, " ")
    return s

def process_tags(tags):
    # Replace common separator with space
    separate_tags = [t.replace('-', ' ').replace('/', ' ').lower() for t in tags]
    # Some tags are still in string format separated by comma, separate them
    no_comma_tags = []
    for t in separate_tags:
            if "," in t:
                split_tags= t.split(',')
                for st in split_tags:
                        no_comma_tags.append(st)
            else:
                no_comma_tags.append(t)

    
    #Lowercase all tags
    no_comma_tags = [t.lower() for t in no_comma_tags]
    
    #Remove special characters from tags
    to_be_removed_char = [';', ':', '!', '*', '(',')','/']
    no_special_tags = [remove_chars(t,to_be_removed_char) for t in no_comma_tags]
    return no_special_tags

combine_df['Genres'] = combine_df['Genres'].progress_apply(process_tags)

Processing Tags:   0%|          | 0/78774 [00:00<?, ?it/s]

In [ ]:
#Get unique tags list
unique_tags = set()
for t in combine_df['Genres']:
    unique_tags.update(t)
unique_tags = list(unique_tags)

tags_to_replace = {}

#Even with l=en in the API call some tags are in native language, Translate unique tags to english
for t in tqdm(unique_tags,desc = 'Translating tags'):
        translated_text = GoogleTranslator(source='auto', target='en').translate(t)
        #Single is translated to one for most languages, replace it
        translated_text = translated_text.replace("one ","single ")
        tags_to_replace[t] = translated_text
        
#Apply translations
tqdm.pandas(desc="Processing Tags")
def replace_translation(tags):
    translated=[]
    for t in tags:
        translated.append(tags_to_replace.get(t))
    return translated
combine_df['Genres'] = combine_df['Genres'].progress_apply(replace_translation)

#### Compute similarity between tags to remove duplicates or near duplicates

In [ ]:
similar_tags = {}
def is_similar(a, b):
    return SequenceMatcher(None, a, b).ratio() > 0.7

for i,t1 in enumerate(unique_tags):
    for j in range(i+1,len(unique_tags)):
        if is_similar(t1, unique_tags[j]):
            similar_tags[t1] = unique_tags[j]

In [19]:
def replace_similar(tags):
    replaced_tags = []
    for t in tags:
        if t in similar_tags:
            replaced_tags.append(similar_tags.get(t))
        else:
            replaced_tags.append(t)
    return replaced_tags
combine_df['Genres'] = combine_df['Genres'].progress_apply(replace_similar)

Processing Tags:   0%|          | 0/78774 [00:00<?, ?it/s]

#### Replace manually mistranslations

In [32]:
manual_replace_dict={'physical education': 'fighting',
                     'Strategy':'strategy',
                     'fighters':'fighting',
                     'role':'roleplaying',
                     'racing games':'racing',
                     'working with sound': 'audio production',
                     'share':'shared screen',
                     'massive multiplayer online mmo':'mmo',
                     'strategic':'strategy',
                     'Leisure':'hobby',
                     'sexual content': 'mature content',
                     'massively multiplayer online':'mmo',
                     'episodic':'visual novel',
                     'multiplayer games': 'multiplayer',
                     'exposed':'mature content',
                     'casual games':'casual',
                     'aventura':'adventure',
                     'adventure games':'adventure',
                     'aktion':'action',
                     'independent':'indie',
                     'Sexual content':'mature content',
                     'rpg role playing game':'role-playing games',
                     'massive multiplayer':'mmo',
                     'race':'racing',
                     'strategie':'strategy',
                     'recreational':'casual',
                     'desporto':'sports',
                     'independent producer':'indie',
                     'rpg':'role-playing games',
                     'role play':'role-playing games',
                     'for free':'free to play',
                     'strategies':'strategy',
                     'massively multiplayer':'mmo' ,
                     'estrategia':'strategy',
                     'simulationen':'simulation',
                     'ryo':'strategy'
                    }

In [33]:
def replace_manual(tags):
    replaced_tags = []
    for t in tags:
        if t in manual_replace_dict:
            replaced_tags.append(manual_replace_dict.get(t))
        else:
            replaced_tags.append(t)
    return replaced_tags
combine_df['Genres'] = combine_df['Genres'].progress_apply(replace_manual)

Processing Tags:   0%|          | 0/78774 [00:00<?, ?it/s]

#### Insert cleaned tags in mongodb

In [36]:
%%script False
collection = db['Clean_tags_games']

# Convert df to list of dict and insert into MongoDB
data_dict = combine_df.to_dict("records")
collection.insert_many(data_dict)

print("Data inserted into MongoDB.")

Couldn't find program: 'False'


#### Assign tags to cluster

In [11]:
collection = db['Clean_tags_games']
tag_cluster_tf = pd.DataFrame(list(collection.find()))

In [12]:
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(tag_cluster_tf['Genres'])


In [13]:
genre_columns = mlb.classes_

In [14]:
# Create a df from the one-hot encoded genres
genres_df = pd.DataFrame(genres_encoded, columns=genre_columns)
cluster_df = tag_cluster_tf.drop(['Genres','Categories','Combined Tags','_id', 'AppID', 'Name', 'Positive', 'Negative'],axis = 1)


In [15]:
#Min max normalization

cluster_df['Peak CCU'] = (cluster_df['Peak CCU']-cluster_df['Peak CCU'].min())/(cluster_df['Peak CCU'].max()-cluster_df['Peak CCU'].min())
cluster_df['Net Votes'] = (cluster_df['Net Votes']-cluster_df['Net Votes'].min())/(cluster_df['Net Votes'].max()-cluster_df['Net Votes'].min())
# Concatenate the original DataFrame with the new one containing the one-hot encoded genres
cluster_df = pd.concat([cluster_df, genres_df], axis=1)

#### Cluster word embedding - hopkins statistics for cluster tendency

In [178]:
def hopkins_statistic(X):
    X_sample = shuffle(X, random_state=42)[:int(X.shape[0]/10)]  
    nn = NearestNeighbors(n_neighbors=1).fit(X)
    
    # NN distances in the real dataset
    real_distances, _ = nn.kneighbors(X_sample, return_distance=True)
    real_distances_sum = np.sum(real_distances)
    
    #Simulated points with same variance of the dataset
    random_points = np.random.uniform(np.min(X, axis=0), np.max(X, axis=0), X_sample.shape)
    simulated_distances, _ = nn.kneighbors(random_points, return_distance=True)
    simulated_distances_sum = np.sum(simulated_distances)
    H = simulated_distances_sum / (real_distances_sum + simulated_distances_sum)
    return H

H = hopkins_statistic(tag_vectors)
print(H)

0.999999988068642
